In [5]:
%matplotlib inline

import urllib.request
import io
import zipfile

from pathlib import Path
import os

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

import igraph as ig

from itertools import combinations

In [6]:
#instance

inst_="karate"
G = nx.karate_club_graph()

#inst_="sudoku"
#G = nx.sudoku_graph(50)

#inst_="random_tree"
#G = nx.random_tree(50)

#inst_="random_internet_100"
#G = nx.random_internet_as_graph(100)

# erdos_renyi_graph(n, p, seed=None, directed=False)
#G = nx.erdos_renyi_graph(50,0.1)

# barabasi_albert_graph(n, m, seed=None, initial_graph=None)
#G = nx.barabasi_albert_graph()


#print("Node Degree")
#for v in G:
#    print(f"{v:4} {G.degree(v):6}")
#print(G)

In [7]:
nx.is_connected(G)

True

In [8]:
def is_subclique(G, nodelist):
    r'''For each pair of nodes in nodelist whether there is an edge
        if any edge is missing, we know that it's not a subclique.
        if all edges are there, it is a subclique
    '''
    for (u,v) in combinations(nodelist,2):  #check each possible pair
        if not G.has_edge(u,v):
            return False #if any edge is missing we're done
    return True  #if we get to here, then every edge was there.  It's True.

In [9]:
N = nx.number_of_nodes(G)
M = nx.number_of_edges(G)

dm = np.zeros((N,N), dtype=int)

p = nx.shortest_path(G)

for i in range(0,N):
    for j in range(0,N):
        dm[i][j] = len(p[i][j])-1

#G.edges


In [12]:
for i in G:
    Ni = nx.neighbors(G,i)

    lista = []
    for k in Ni:
        lista.append(k)
    
    if(is_subclique(G, lista)):
        print("vizinhanca de %d" %i)


vizinhanca de 7
vizinhanca de 11
vizinhanca de 12
vizinhanca de 14
vizinhanca de 15
vizinhanca de 16
vizinhanca de 17
vizinhanca de 18
vizinhanca de 20
vizinhanca de 21
vizinhanca de 22
vizinhanca de 26


In [14]:
# form1
form_="form1"
method_="mip"

N = nx.number_of_nodes(G)
M = nx.number_of_edges(G)

lb = np.zeros((N), dtype=float)
ub = np.zeros((N), dtype=float)
time = np.zeros((N), dtype=float)
gap = np.zeros((N), dtype=float)
nodes = np.zeros((N), dtype=float)
status = np.zeros((N), dtype=float)

RESULT_PATH   = Path('../result/')

for i in G:

    Ni = nx.neighbors(G,i)

    lista = []
    for k in Ni:
        lista.append(k)
    
    if(is_subclique(G, lista)):
        print("tukey[%d] = 1" %i)
        lb[i] = N - model.objBound
        ub[i] = N - model.objVal
        gap[i] = model.MIPGap
        time[i] = model.Runtime
        nodes[i] = model.NodeCount
        status[i] = tmp
    else:

        model = gp.Model(f"{method_}_{inst_}_{form_}")

        # configurando parametros
        # model.Params.TimeLimit = 60
        model.Params.MIPGap = 1.e-6
        model.Params.Threads = 1
        # model.Params.Presolve = 0
        # model.Params.Cuts = 0

        if (method_=="mip"):
            x = model.addVars(N, vtype=GRB.BINARY, name="x")
        else:
            x = model.addVars(N, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="x")
   
        obj = 0
        for j in G:
            obj += 1 * x[j]
         
        model.setObjective(obj, GRB.MAXIMIZE)
     
        # Turn off display and heuristics
        gp.setParam('OutputFlag', 0)
        gp.setParam('Heuristics', 0)

        constr = 0
        constr += 1 * x[i]
        model.addConstr(constr == 0)

        for u in range(0,N):
            for w in range(u+1,N):
                if dm[u,w] <= N:
                    for s in range(0,N):
                        if (s != u) and (s != w):
                            if (dm[u,s] + dm[s,w] == dm[u,w]):
                                model.addConstr(x[u] + x[w] <= 1 + x[s])

        for u in range(0,N):
            if (u == i):
                Im = nx.maximal_independent_set(G, [i])
                constr = 0
                for k in Im:
                    constr = 1 * x[k]
                model.addConstr(constr <= 1)
            else:
                Im = nx.maximal_independent_set(G, [u])
                constr = 0
                for k in Im:
                    constr = 1 * x[k]
                model.addConstr(constr <= 1 + (len(Im)- 1)*x[u])

        #x[i].Start = 0.0
        #relax = model.relax()
        #relax.optimize()

        #model.write(f"{method_}_{inst_}_{form_}.lp")

        model.optimize()

        tmp = 0
        if model.status == GRB.OPTIMAL:
            tmp = 1
    
        if (method_=="mip"):
            lb[i] = N - model.objBound
            ub[i] = N - model.objVal
            gap[i] = model.MIPGap
            time[i] = model.Runtime
            nodes[i] = model.NodeCount
            status[i] = tmp
        else:
            ub[i] = N - model.objVal
            time[i] = model.Runtime
            status[i] = tmp

        if (method_=="mip"):
            arquivo = open(os.path.join(RESULT_PATH,f"{method_}_tukey_{inst_}_{form_}.txt"),'a')
            tmp = i
            arquivo.write(
                str(tmp)+';'
                +str(round(lb[i],1))+';'
                +str(round(ub[i],1))+';'
                +str(round(gap[i],2))+';'
                +str(round(time[i],2))+';'
                +str(round(nodes[i],1))+';'
                +str(round(status[i],1))+'\n')
            arquivo.close()
        else:
            arquivo = open(os.path.join(RESULT_PATH,f"{method_}_tukey_{inst_}_{form_}.txt"),'a')
            tmp = i
            arquivo.write(
                str(tmp)+';'
                +str(round(ub[i],1))+';'
                +str(round(time[i],2))+';'
                +str(round(status[i],1))+'\n')
            arquivo.close()

tukey[7] = 1
tukey[11] = 1
tukey[12] = 1
tukey[14] = 1
tukey[15] = 1
tukey[16] = 1
tukey[17] = 1
tukey[18] = 1
tukey[20] = 1
tukey[21] = 1
tukey[22] = 1
tukey[26] = 1


In [28]:
#t = 0
#for i in ub:
#    print("tukey[%d] =" %t, end=" ")
#    print("%g" %i)
#    t = t+1